# Verify Option 8: Full Taxation of Social Security Benefits

This notebook verifies that Option 8 results in `taxable_social_security == social_security` at the national aggregate level.

In [1]:
from policyengine_us import Microsimulation
from src.reforms import get_option8_reform
import numpy as np
import pandas as pd

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup Reform and Simulation

In [2]:
# Create Option 8 reform
reform = get_option8_reform()

# Create microsimulation
print("Loading microsimulation data...")
sim = Microsimulation(reform=reform)
print("Done!")

Loading microsimulation data...
Done!


## Calculate Aggregate Values for 2026

In [3]:
year = 2026

print(f"Calculating aggregate values for {year}...")
social_security = sim.calculate("social_security", year)
taxable_social_security = sim.calculate("taxable_social_security", year)

# Aggregate totals
total_ss = social_security.sum()
total_taxable_ss = taxable_social_security.sum()
difference = total_ss - total_taxable_ss
percentage_taxable = (total_taxable_ss / total_ss * 100) if total_ss > 0 else 0

print(f"\nAggregate Results for {year}:")
print(f"  Total Social Security Benefits:     ${total_ss:>20,.0f}")
print(f"  Total Taxable Social Security:      ${total_taxable_ss:>20,.0f}")
print(f"  Difference:                         ${difference:>20,.0f}")
print(f"  Percentage Taxable:                 {percentage_taxable:>20.2f}%")

Calculating aggregate values for 2026...

Aggregate Results for 2026:
  Total Social Security Benefits:     $   1,494,708,375,943
  Total Taxable Social Security:      $   1,493,096,104,367
  Difference:                         $       1,612,271,577
  Percentage Taxable:                                99.89%


## Investigate Households Where Taxable SS ≠ SS

In [4]:
# Find households with SS benefits
has_ss = social_security > 0

# Calculate the ratio
ratio = np.divide(
    taxable_social_security,
    social_security,
    out=np.zeros_like(taxable_social_security, dtype=float),
    where=social_security > 0
)

print(f"\nStatistics for households with SS benefits:")
print(f"  Total households with SS: {has_ss.sum():,}")
print(f"  Total SS benefits:        ${social_security[has_ss].sum():,.0f}")
print(f"  Total taxable SS:         ${taxable_social_security[has_ss].sum():,.0f}")
print(f"  Overall percentage:       {(taxable_social_security[has_ss].sum() / social_security[has_ss].sum() * 100):.4f}%")


Statistics for households with SS benefits:
  Total households with SS: 65,956,134.367887095
  Total SS benefits:        $1,494,708,375,943
  Total taxable SS:         $1,467,136,474,093
  Overall percentage:       98.1554%


In [5]:
# Find households where taxable != total (allowing for small floating point errors)
not_fully_taxable = has_ss & (np.abs(taxable_social_security - social_security) > 0.01)

print(f"\nHouseholds where taxable_ss != ss:")
print(f"  Count:                    {not_fully_taxable.sum():,}")
print(f"  Their SS benefits:        ${social_security[not_fully_taxable].sum():,.0f}")
print(f"  Their taxable SS:         ${taxable_social_security[not_fully_taxable].sum():,.0f}")
print(f"  Difference:               ${(social_security[not_fully_taxable].sum() - taxable_social_security[not_fully_taxable].sum()):,.0f}")

# Distribution of ratios
problem_ratios = ratio[not_fully_taxable]
if len(problem_ratios) > 0:
    print(f"\nRatio distribution for problem households:")
    print(f"  Min ratio:                {problem_ratios.min():.4f}")
    print(f"  Max ratio:                {problem_ratios.max():.4f}")
    print(f"  Mean ratio:               {problem_ratios.mean():.4f}")
    print(f"  Median ratio:             {np.median(problem_ratios):.4f}")


Households where taxable_ss != ss:
  Count:                    165
  Their SS benefits:        $33,780,326,066
  Their taxable SS:         $6,208,421,894
  Difference:               $27,571,904,172

Ratio distribution for problem households:
  Min ratio:                0.0000
  Max ratio:                7.6572
  Mean ratio:               0.3653
  Median ratio:             0.0000


## Check Reform Parameters

In [6]:
print(f"\nReform parameter values:")
params = reform.parameter_values
ss_params = {k: v for k, v in params.items() if "social_security" in str(k).lower()}

for key, value in sorted(ss_params.items()):
    print(f"  {key}")
    print(f"    {value}")


Reform parameter values:
  gov.irs.social_security.taxability.combined_income_ss_fraction
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.benefit_cap
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.bracket
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.excess
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.base.benefit_cap
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.base.excess
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.HEAD_OF_HOUSEHOLD
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.JOINT
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.SEPARATE
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.SING

## Summary

In [7]:
print("\n" + "="*80)
if abs(difference) < 1000:
    print("[PASS] Option 8 correctly taxes 100% of Social Security benefits nationwide")
    print("       (within rounding tolerance)")
else:
    print("[FAIL] Option 8 does NOT tax 100% of Social Security benefits")
    print(f"       Difference of ${difference:,.0f} ({100-percentage_taxable:.2f}% untaxed)")
    print(f"       This affects {not_fully_taxable.sum():,} households")
print("="*80)


[FAIL] Option 8 does NOT tax 100% of Social Security benefits
       Difference of $1,612,271,577 (0.11% untaxed)
       This affects 165 households
